The model is designed to analyze and classify emotional connotation of texts

BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# model used is Bert
model_name = "nateraw/bert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# customer input to be analyzed
text = input("Please, enter customer review to be analyzed")
print(f"\nAnalyzing: {text}\n")
inputs = tokenizer(text, return_tensors="pt")


with torch.no_grad():
    outputs = model(**inputs)

#softmax to get probabilities
probs = F.softmax(outputs.logits, dim=-1)

# emotion labels
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

# emotion with scores
for label, score in zip(labels, probs[0]):
    print(f"{label}: {score.item():.4f}")

The product has arrived damaged and not ina  good state!!!

Analyzing: The product has arrived damaged and not ina  good state!!!

sadness: 0.9587
joy: 0.0096
love: 0.0018
anger: 0.0188
fear: 0.0098
surprise: 0.0013


RoBERTa with 4 emotions

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# model used is RoBERTa
model_name = "cardiffnlp/twitter-roberta-base-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ['anger', 'joy', 'optimism', 'sadness']

# customer input to be analyzed
text = input("Please, enter customer review to be analyzed ")
print(f"\nAnalyzing: {text}\n")
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

with torch.no_grad():
    outputs = model(**inputs)

#softmax to get probabilities
probs = F.softmax(outputs.logits, dim=-1)

# emotion with scores
for label, score in zip(labels, probs[0]):
    print(f"{label}: {score.item():.4f}")

Please, enter customer review to be analyzed This product has been not what I've expected. It makes noises and delivery took ages. 

Analyzing: This product has been not what I've expected. It makes noises and delivery took ages. 

sadness: 0.8705
joy: 0.0101
love: 0.0118
anger: 0.1076


RoBERTa multiple input, 6 emotions

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# model used is RoBERTa with wider variety of emotions
model_name = "bhadresh-savani/roberta-base-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

reviews = [
    "I feel so disappointed and tired of everything.",
    "What an amazing experience, I loved it!",
    "This makes me furious. Terrible service.",
    "I'm scared about the results.",
    "Such a joyful and bright day!",
    "My heart is full of love for this place.",
    "I'm overwhelmed and deeply sad.",
    "They treated me with no respect, I'm angry.",
    "I'm okay, just confused.",
    "A surprise party? I didn’t expect that!"
]

# Analyze all reviews
def analyze_reviews(reviews):
    sadness_reviews = []
    anger_reviews = []

    for i, review in enumerate(reviews):
        inputs = tokenizer(review, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        probs_dict = {label: probs[0][idx].item() for idx, label in enumerate(labels)}

        # Thresholds can be adjusted
        if probs_dict["sadness"] >= 0.4:
            sadness_reviews.append((review, probs_dict["sadness"]))
        if probs_dict["anger"] >= 0.4:
            anger_reviews.append((review, probs_dict["anger"]))

        # Print probabilities for each review
        print(f"\nReview {i+1}: {review}")
        for label, score in probs_dict.items():
            print(f"  {label}: {score:.4f}")

    # Summary
    print("\n--- Reviews related to Sadness ---")
    for text, score in sadness_reviews:
        print(f"  - [{score:.2f}] {text}")

    print("\n--- Reviews related to Anger ---")
    for text, score in anger_reviews:
        print(f"  - [{score:.2f}] {text}")

# Run analysis
analyze_reviews(reviews)

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]


Review 1: I feel so disappointed and tired of everything.
  sadness: 0.9975
  joy: 0.0006
  love: 0.0004
  anger: 0.0009
  fear: 0.0004
  surprise: 0.0003

Review 2: What an amazing experience, I loved it!
  sadness: 0.0003
  joy: 0.9935
  love: 0.0024
  anger: 0.0005
  fear: 0.0004
  surprise: 0.0029

Review 3: This makes me furious. Terrible service.
  sadness: 0.0011
  joy: 0.0011
  love: 0.0005
  anger: 0.9947
  fear: 0.0020
  surprise: 0.0006

Review 4: I'm scared about the results.
  sadness: 0.0021
  joy: 0.0027
  love: 0.0009
  anger: 0.0100
  fear: 0.9832
  surprise: 0.0012

Review 5: Such a joyful and bright day!
  sadness: 0.0002
  joy: 0.9984
  love: 0.0004
  anger: 0.0003
  fear: 0.0004
  surprise: 0.0003

Review 6: My heart is full of love for this place.
  sadness: 0.0050
  joy: 0.5936
  love: 0.3950
  anger: 0.0035
  fear: 0.0012
  surprise: 0.0017

Review 7: I'm overwhelmed and deeply sad.
  sadness: 0.0091
  joy: 0.0123
  love: 0.0015
  anger: 0.0142
  fear: 0.8720
 

In [ ]:
#added GUI

In [18]:
!pip install gradio transformers torch --quiet

import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name = "nateraw/bert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

def analyze_emotions(text_block):
    reviews = [line.strip() for line in text_block.strip().split('\n') if line.strip()]

    if not reviews:
        return "Please enter at least one customer review."

    results = []
    for review in reviews:
        inputs = tokenizer(review, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        scores = probs[0].tolist()

        # dominant emotion
        max_index = scores.index(max(scores))
        dominant_emotion = labels[max_index]
        emotion_scores = ", ".join(f"{label}: {scores[i]:.4f}" for i, label in enumerate(labels))

        # highlight review if dominant emotion is anger or sadness
        if dominant_emotion in ["anger", "sadness"]:
            review_output = f"🔴 [High {dominant_emotion.upper()}] Review: {review}\n{emotion_scores}\n"
        else:
            review_output = f"Review: {review}\n{emotion_scores}\n"

        results.append(review_output)

    return "\n".join(results)

iface = gr.Interface(
    fn=analyze_emotions,
    inputs=gr.Textbox(
        lines=10,
        placeholder="Enter one review per line...",
        label="Customer Reviews Input"
    ),
    outputs=gr.Textbox(
        label="Emotion Analysis Results"
    ),
    title="Emotion Analyzer",
    description="Enter multiple reviews (one per line). Reviews with dominant *anger* or *sadness* will be highlighted."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://246cc20736dcaf9745.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Test Accuracy

In [20]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
model_name = "nateraw/bert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

test_texts = [
    "I am extremely happy with how this product works; it exceeded all my expectations and makes my life easier every day.",
    "This experience was really disappointing and bad; the product broke within days and customer support was unhelpful.",
    "I absolutely love using this every single day because it fits perfectly into my routine and brings me great joy.",
    "I'm feeling quite scared about what might happen next, given the uncertain circumstances and lack of clear answers.",
    "What a surprise it was to receive this gift today; it was beautifully wrapped and truly brightened my entire day.",
    "I'm very angry about the terrible customer service; they ignored my complaints and refused to provide a refund."
]

true_labels = ['joy', 'sadness', 'love', 'fear', 'surprise', 'anger']

predicted_labels = []

model.eval()
with torch.no_grad():
    for text in test_texts:
        inputs = tokenizer(text, return_tensors="pt")
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        pred_id = torch.argmax(probs, dim=-1).item()
        predicted_labels.append(id2label[pred_id])

# metrics
true_ids = [label2id[label] for label in true_labels]
pred_ids = [label2id[label] for label in predicted_labels]

acc = accuracy_score(true_ids, pred_ids)
precision, recall, f1, _ = precision_recall_fscore_support(true_ids, pred_ids, average='macro')

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(true_ids, pred_ids, target_names=labels))

# print predictions side by side
print("\nPredictions:")
for text, true_label, pred_label in zip(test_texts, true_labels, predicted_labels):
    print(f"Text: {text}")
    print(f"True: {true_label} | Predicted: {pred_label}")
    print()

Accuracy: 0.6667
Precision: 0.5556, Recall: 0.6667, F1-score: 0.5833

Classification Report:
              precision    recall  f1-score   support

     sadness       1.00      1.00      1.00         1
         joy       0.33      1.00      0.50         1
        love       0.00      0.00      0.00         1
       anger       1.00      1.00      1.00         1
        fear       1.00      1.00      1.00         1
    surprise       0.00      0.00      0.00         1

    accuracy                           0.67         6
   macro avg       0.56      0.67      0.58         6
weighted avg       0.56      0.67      0.58         6


Predictions:
Text: I am extremely happy with how this product works; it exceeded all my expectations and makes my life easier every day.
True: joy | Predicted: joy

Text: This experience was really disappointing and bad; the product broke within days and customer support was unhelpful.
True: sadness | Predicted: sadness

Text: I absolutely love using this every

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m